# Binder-ready Interactive Q&A Notebook
This notebook loads your `master.xlsx` file (all relevant sheets) and allows interactive fuzzy-search Q&A.

In [ ]:
import pandas as pd
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.display import display
import os

# -------------------------------
# Step 1: Load Master Excel File
# -------------------------------

file_path = 'master.xlsx'  # Make sure this file is in the repo root

sheet_names = [
    'Deposit', 'CRM', 'CKYC', 'Advance', 'Locker', 'SVS', 'Teller',
    'RTGS NEFT Bills BG', 'TXN related', 'Lien Related', 'Office AC',
    'Others', 'VNDRINVC_SSAM', 'TDS Related', 'CERSAI'
]

all_dfs = []

try:
    xls = pd.ExcelFile(file_path)
    for sheet in sheet_names:
        if sheet in xls.sheet_names:
            temp_df = pd.read_excel(file_path, sheet_name=sheet)
            temp_df['Sheet'] = sheet  # track source sheet
            all_dfs.append(temp_df)
        else:
            print(f"Sheet '{sheet}' not found in file.")
    if all_dfs:
        df = pd.concat(all_dfs, ignore_index=True)
        print("Master file loaded successfully!")
        print("Columns:", df.columns.tolist())
    else:
        print("No valid sheets found. Master file is empty.")
        df = pd.DataFrame()
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
    df = pd.DataFrame()

# Normalize column names
df.columns = [col.strip() for col in df.columns]

required_columns = ['Question/Issue', 'Solution']
if not all(col in df.columns for col in required_columns):
    print(f"Master file must have columns: {required_columns}")
    df = pd.DataFrame(columns=required_columns)

# -------------------------------
# Step 2: Define Fuzzy Search Function
# -------------------------------

def get_solution_fuzzy(user_question, df, threshold=70):
    if df.empty:
        return "Master file is empty or not loaded correctly."
    
    questions = df['Question/Issue'].astype(str).tolist()
    matches = process.extract(user_question, questions, limit=5)
    best_matches = [match for match in matches if match[1] >= threshold]
    
    if not best_matches:
        return "No matching solution found in master file."
    
    solutions = []
    for match in best_matches:
        matched_question = match[0]
        solution_row = df[df['Question/Issue'] == matched_question][['Solution', 'Sheet']].values.tolist()
        solutions.extend(solution_row)
    
    formatted_solutions = [f"{sol[0]} (Sheet: {sol[1]})" for sol in solutions]
    return formatted_solutions

# -------------------------------
# Step 3: Create Interactive Widgets
# -------------------------------

question_box = widgets.Text(
    description="Your Question:",
    placeholder="Type your question here"
)
output_box = widgets.Output()
submit_btn = widgets.Button(description="Get Solution")

def on_submit(b):
    user_question = question_box.value
    response = get_solution_fuzzy(user_question, df)
    with output_box:
        output_box.clear_output()
        if isinstance(response, list):
            for sol in response:
                print("-", sol)
        else:
            print(response)

submit_btn.on_click(on_submit)
display(question_box, submit_btn, output_box)